In [278]:
using LinearAlgebra
using TensorOperations
using Plots

### Canonical Forms

$\quad$ Before going into the implementation of DMRG, we first build a function that transforms an arbitrary MPS into a convenient canonical form.

$\quad$ We start by creating a random MPS with $N$ sites, physical dimension $d$ (for simplicity, equal in all sites) and bond dimension $D$ (for simplicity, equal in all bonds, except for the outer bonds, which are set trivial), as depicted below.

<img src="./img/MPS.svg" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>

In [280]:
# parameters 
N = 10
d = 3
D = 20

# random MPS
#=
    Order of legs: left-bottom-right
    Note: this is the conventional order used for MPSs in the code.
=#

Mrand = []
push!(Mrand, rand(1, d, D))
for l in 2:N-1
    push!(Mrand, rand(D, d, D))
end
push!(Mrand, rand(D, d, 1))

10-element Vector{Any}:
 [0.7873314361666243 0.6700993490838282 0.4612319243579672;;; 0.8643381144071688 0.6591711822597706 0.8483063423551799;;; 0.7220327850783638 0.3461381184677358 0.7953659418454534;;; … ;;; 0.977016818686399 0.33578002264978835 0.05811921239622708;;; 0.3488085009567887 0.13514670666250006 0.7498791852812725;;; 0.16908120566430584 0.6470369270620652 0.16203780248556776]
 [0.6529169793605097 0.9287335406760134 0.06375746191703624; 0.7323633205698474 0.23816836019144072 0.20822970475595315; … ; 0.49536020259576996 0.3150342587362074 0.7347248812468771; 0.6270370528858229 0.635039107302168 0.7464090973994506;;; 0.20043279469529807 0.5480963732614941 0.3987698976388744; 0.7327820431896006 0.4510065840547315 0.7602438713192524; … ; 0.8766476243547221 0.16904022098612992 0.7168043631691124; 0.3164073018963376 0.2987381803189215 0.6323115958747844;;; 0.6008151060787017 0.18492840844195935 0.007186773070864572; 0.5429982900583028 0.1369078378840931 0.3053300774083135; … ; 

$\quad$ We now define a function that takes an MPS as input and transforms it into left canonical form, normalizing it. The output of this function is an equivalent description of the input MPS (apart from the normalization) where all tensors are left normalized. This is achieved using SVD, following the procedure depicted below.

<img src="./img/LeftCanonical_procedure.svg" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>

<u>Clarifying note</u>: the dashed line is a graphical notation to symbolize that the tensor is normalized with respect to those indices. For example, if a 3-leg tensor $U^{\alpha,\sigma,\beta}$ has a dashed line between legs $\alpha$ and $\sigma$, we have
$$
\sum_{\alpha,\sigma} \left( U^{\alpha,\sigma,\beta} \right)^* U^{\alpha,\sigma,\beta'} = \delta_{\beta,\beta'}, 
$$
or, in matrix notation,
$$
U^\dagger U = I,
$$
where the basis $\{ (\alpha,\sigma),\beta \}$ is implicit.

In [ ]:
function left_canonical(M)
    Mcopy = copy(M)
    N = length(Mcopy)

    for n in 1:N
        Taux = Mcopy[n]
        Taux = reshape(Taux, (size(Taux, 1)*size(Taux, 2), size(Taux, 3)))
        
        F = svd(Taux)
        U, S, Vdag = F
        Mcopy[n] = reshape(U, (size(Mcopy[n], 1), size(Mcopy[n], 2), size(U, 2)))
        SVdag = Diagonal(S) * transpose(Vdag)

        if n < N
            @tensor result[i, k, m] := SVdag[i, j] * Mcopy[n+1][j, k, m]
            Mcopy[n+1] = result
        end
    end 
    
    return Mcopy
end

left_canonical (generic function with 1 method)

$\quad$ Let us now test the use of the previous function. For that matter, we will check if, after applying it, all tensors become left normalized, i.e.,

<img src="./img/left_normalization.svg" width="30%" style="display: block; margin-left: auto; margin-right: auto;"/>

In [7]:
Mleft = left_canonical(Mrand)
for n in 1:N
    Mdag = conj.(permutedims(Mleft[n], (3, 2, 1)))
    @tensor result[i, l] := Mdag[i, j, k] * Mleft[n][k, j, l]
    MdagM = result
    I = Diagonal(ones(size(Mleft[n], 3)))
    diff = maximum(abs.(MdagM .- I))
    println("l = $n, : max(|M[n] * M[n]^† - I|) = $diff")
end

l = 1, : max(|M[n] * M[n]^† - I|) = 1.4432899320127035e-15
l = 2, : max(|M[n] * M[n]^† - I|) = 1.9984014443252818e-15
l = 3, : max(|M[n] * M[n]^† - I|) = 2.4424906541753444e-15
l = 4, : max(|M[n] * M[n]^† - I|) = 1.6653345369377348e-15
l = 5, : max(|M[n] * M[n]^† - I|) = 1.5543122344752192e-15
l = 6, : max(|M[n] * M[n]^† - I|) = 1.3322676295501878e-15
l = 7, : max(|M[n] * M[n]^† - I|) = 2.886579864025407e-15
l = 8, : max(|M[n] * M[n]^† - I|) = 2.4424906541753444e-15
l = 9, : max(|M[n] * M[n]^† - I|) = 1.9984014443252818e-15
l = 10, : max(|M[n] * M[n]^† - I|) = 4.440892098500626e-16


In [82]:
function right_canonical(M)
    Mcopy = copy(M)
    N = length(Mcopy)

    for n in N:-1:2
        Taux = Mcopy[n]
        Taux = reshape(Taux, (size(Taux, 1), size(Taux, 2)*size(Taux, 3)))

        F = svd(Taux)
        U, S, Vdag = F
        Vdag = transpose(Vdag) 

        Mcopy[n] = reshape(Vdag, (size(Vdag, 1), size(Mcopy[n], 2), size(Mcopy[n], 3)))
        US = U * Diagonal(S)

        if n > 0
            @tensor result[i, j, l] := Mcopy[n-1][i, j, k] * US[k, l]
            Mcopy[n-1] = result
        end
    end 
    
    return Mcopy
end

right_canonical (generic function with 1 method)

In [84]:
Mright = right_canonical(Mrand)
for n in 1:N
    Mdag = conj.(permutedims(Mright[n], (3, 2, 1)))
    @tensor result[i, l] := Mright[n][i, j, k] * Mdag[k, j, l]
    MMdag = result
    I = Diagonal(ones(size(Mright[n], 1)))
    diff = maximum(abs.(MMdag .- I))
    println("l = $n, : max(|M[l] · M[l]^† - I|) = $diff")
end

l = 1, : max(|M[l] · M[l]^† - I|) = 1.6552893747887004e22
l = 2, : max(|M[l] · M[l]^† - I|) = 1.5543122344752192e-15
l = 3, : max(|M[l] · M[l]^† - I|) = 1.1102230246251565e-15
l = 4, : max(|M[l] · M[l]^† - I|) = 1.7763568394002505e-15
l = 5, : max(|M[l] · M[l]^† - I|) = 2.220446049250313e-15
l = 6, : max(|M[l] · M[l]^† - I|) = 2.886579864025407e-15
l = 7, : max(|M[l] · M[l]^† - I|) = 1.5543122344752192e-15
l = 8, : max(|M[l] · M[l]^† - I|) = 2.3314683517128287e-15
l = 9, : max(|M[l] · M[l]^† - I|) = 1.1102230246251565e-15
l = 10, : max(|M[l] · M[l]^† - I|) = 6.661338147750939e-16


$\quad$ As a final remark, we note that in tensor network algorithms starting with a random MPS with trivial outer bonds, it is standard practice to apply both 'LeftCanonical' and 'RightCanonical' functions once. This ensures two things: i) that the random MPS is normalized; ii) that some (redundant) bond dimensions (typically closer to the end sites) are trivially truncated due to the thin SVD.

In [90]:
println("Dimensions of Mrand:")
for n in 1:N
    println("n = $n : $(size(Mrand[n]))")
end 

Mrand2 = left_canonical(Mrand)
println("\nAfter LeftCanonical function:")
for n in 1:N
    println("n = $n  : $(size(Mrand2[n]))")
end

Mrand3 = right_canonical(Mrand2)
println("\nAfter LeftCanonical and RightCanonical functions:")
for n in 1:N
    println("n = $n  : $(size(Mrand3[n]))")
end




Dimensions of Mrand:
n = 1 : (1, 3, 20)
n = 2 : (20, 3, 20)
n = 3 : (20, 3, 20)
n = 4 : (20, 3, 20)
n = 5 : (20, 3, 20)
n = 6 : (20, 3, 20)
n = 7 : (20, 3, 20)
n = 8 : (20, 3, 20)
n = 9 : (20, 3, 20)
n = 10 : (20, 3, 1)

After LeftCanonical function:
n = 1  : (1, 3, 3)
n = 2  : (3, 3, 9)
n = 3  : (9, 3, 20)
n = 4  : (20, 3, 20)
n = 5  : (20, 3, 20)
n = 6  : (20, 3, 20)
n = 7  : (20, 3, 20)
n = 8  : (20, 3, 20)
n = 9  : (20, 3, 20)
n = 10  : (20, 3, 1)

After LeftCanonical and RightCanonical functions:
n = 1  : (1, 3, 3)
n = 2  : (3, 3, 9)
n = 3  : (9, 3, 20)
n = 4  : (20, 3, 20)
n = 5  : (20, 3, 20)
n = 6  : (20, 3, 20)
n = 7  : (20, 3, 20)
n = 8  : (20, 3, 9)
n = 9  : (9, 3, 3)
n = 10  : (3, 3, 1)


# MPOs of Hamiltonians

$\quad$ In this section, we show how to build the MPO of a model Hamiltonian, following the strategy explained in the manuscript. For concreteness, we focus on Hamiltonians of 1D quantum spin chains with open boundary conditions. A general representation of such an Hamiltonian MPO is depicted below, where $N$ is the number of sites in the chain and $d$ is the corresponding local dimension ($d=2s+1$ for a spin-$s$ system).

<img src="./img/MPO_Hamiltonian.svg" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>
<br>

For clarity purposes, we consider four different examples, which should illustrate how to do it for a general model.

## i) XY model

$\quad$ The Hamiltonian of an XY open-ended chain is given by
$$
\hat{\mathcal{H}}_\text{XY} = -\sum_{l=0}^{N-2} \left( \hat{\sigma}^x_l \hat{\sigma}^x_{l+1} + \hat{\sigma}^y_l \hat{\sigma}^y_{l+1} \right),
$$
where $N$ is the number of $s=1/2$ spins and $\hat{\sigma}^{x/y}_l$ are the $x$/$y$ spin-1/2 operators at site $l$.

$\quad$ Straightforward manipulation gives
$$
\hat{\mathcal{H}}_\text{XY} = -\frac{1}{2} \sum_{l=0}^{N-2} \left( \hat{\sigma}^+_l \hat{\sigma}^-_{l+1} + \hat{\sigma}^-_l \hat{\sigma}^+_{l+1} \right),
$$
where $\hat{\sigma}^\pm_l = \hat{\sigma}^x_l \pm \mathrm{i} \hat{\sigma}^y_l$ are the spin ladder operators.

In [106]:
function xy_mpo(N)
    sp = zeros((2, 2))
    sp[1, 2] = 1

    sm = zeros((2, 2))
    sm[2, 1] = 1

    I2 = Diagonal(ones(2, 2))

    Hl = zeros((4, 2, 4, 2))
    Hl[1,:,1,:] = I2
    Hl[2,:,1,:] = sm
    Hl[3,:,1,:] = sp
    Hl[4,:,2,:] = -0.5*sp
    Hl[4,:,3,:] = -0.5*sm
    Hl[4,:,4,:] = I2

    H = [Hl for l in 1:N]
    H[1] = Hl[4:4,:,:,:]
    H[N] = Hl[:,:,1:1,:]

    return H
end

xy_mpo (generic function with 1 method)

$\quad$ For completeness, let us now show how to contract the Hamiltonian MPO to obtain the Hamiltonian matrix (in the physical basis), which can then be compared to that obtained by brute-force means. It must be noted that this procedure is not desired (and never done) in a tensor network algorithm, as the dimension of the Hamiltonian matrix scales exponentially with $N$, so that it can only be obtained for small $N$ (this remark is actually illustrative of how efficient MPOs are in the task of enconding the information of a model Hamiltonian). Here, we only do it to convince ourselves that the Hamiltonian MPO was properly constructed.

In [ ]:
function Hmat_xy(N)
    sp = zeros((2, 2))
    sp[1, 2] = 1

    sm = zeros((2, 2))
    sm[2, 1] = 1

    I2 = Diagonal(ones((2, 2)))

    H = zeros((2^N, 2^N))
    for n in 1:N-1
        Ileft = Diagonal(ones((2^(n-1), 2^(n-1))))
        Hmid = (-0.5 * kron(sp, sm)) + (-0.5 * kron(sm, sp))
        Iright = Diagonal(ones((2^(N-n-1), 2^(N-n-1))))
        H += kron(kron(Ileft, Hmid), Iright)
    end

    return H
end


Hmat_xy (generic function with 1 method)

In [ ]:
N = 5
H_mpo = xy_mpo(N)

for n in 1:N
    if n == 1
        Taux = H_mpo[n][1,:,:,:]
    else
        @tensor result[i,k,l,m,o] := Taux[i,j,k] * H_mpo[n][j,l,m,o]
        Taux = result
        Taux = permutedims(Taux, (1, 3, 4, 2, 5))
        Taux = reshape(Taux, (size(Taux, 1)*size(Taux, 2), 
                              size(Taux, 3), 
                              size(Taux, 4)*size(Taux, 5)))
    end
end

Hmat = Taux[:, 1, :]
Hmat2 = Hmat_xy(N)
H_diff = maximum(abs.(Hmat .- Hmat2))

println("difference between MPO and brute-force Hamiltonians = $H_diff")

difference between MPO and brute-force Hamiltonians = 0.0


In [216]:
function Heis_MPO(N, J, h)
    # spin-1 operators (bottom-top)
    ## S^z_l
    Sz = zeros((3,3))
    Sz[1,1] = 1
    Sz[3,3] = -1
    ## S^+_l
    Sp = zeros((3,3))
    Sp[1,2] = sqrt(2)
    Sp[2,3] = sqrt(2)
    ## S^-_l
    Sm = zeros((3,3))
    Sm[2,1] = sqrt(2)
    Sm[3,2] = sqrt(2)
    ## I_l
    I3 = Diagonal(ones((3, 3)))
    
    # MPO Hamiltonian (left-bottom-right-top)
    ## H[l]
    Hl = zeros((5,3,5,3))
    Hl[1,:,1,:] = I3
    Hl[2,:,1,:] = Sz
    Hl[3,:,1,:] = Sm
    Hl[4,:,1,:] = Sp
    Hl[5,:,1,:] = -h*Sz
    Hl[5,:,2,:] = J*Sz
    Hl[5,:,3,:] = J/2*Sp
    Hl[5,:,4,:] = J/2*Sm
    Hl[5,:,5,:] = I3
    
    ## H
    H = [Hl for l in 1:N]
    H[1] = Hl[5:5,:,:,:]
    H[N] = Hl[:,:,1:1,:]
    
    return H
end

Heis_MPO (generic function with 1 method)

## ii) Spin-1 Heisenberg model with Zeeman term

$\quad$ The Hamiltonian for an open-ended Heisenberg chain of $N$ $s=1$ spins, with a Zeeman term, is given by
$$
\hat{\mathcal{H}}_\text{Heis} = J \sum_{l=0}^{N-2} \hat{\vec{S}}_l \cdot \hat{\vec{S}}_{l+1} - h \sum_{l=0}^{N-1} \hat{S}^z_l,
$$
where $J$ and $h$ are model parameters, and $\hat{\vec{S}}_l = (\hat{S}^x_l, \hat{S}^y_l, \hat{S}^z_l)$ is the vector of spin-1 operators at site $l$.

$\quad$ Straightforward manipulation leads to
$$
\hat{\mathcal{H}}_\text{Heis} = J \sum_{l=0}^{N-2} \left( \hat{S}^z_l \hat{S}^z_{l+1} + \frac{1}{2} \hat{S}^+_l \hat{S}^-_{l+1} + \frac{1}{2} \hat{S}^-_l \hat{S}^+_{l+1} \right) - h \sum_{l=0}^{N-1} \hat{S}^z_l,
$$
where $\hat{S}^\pm_l = \hat{S}^x_l \pm \mathrm{i} \hat{S}^y_l$ are the spin ladder operators.

In [251]:
function Hmat_Heis(N,J,h)
    # spin-1 operators
    ## S^z_l
    Sz = zeros((3,3))
    Sz[1,1] = 1
    Sz[3,3] = -1
    ## S^+_l
    Sp = zeros((3,3))
    Sp[1,2] = sqrt(2)
    Sp[2,3] = sqrt(2)
    ## S^-_l
    Sm = zeros((3,3))
    Sm[2,1] = sqrt(2)
    Sm[3,2] = sqrt(2)
    ## I_l
    I3 = Diagonal(ones((3, 3)))
    
    # Hamiltonian matrix
    H = zeros((3^N,3^N))
    for n in 1:N-1
        Ileft = Diagonal(ones((3^(n-1), 3^(n-1)))) #I_0 x I_1 x ... I_{l-1}
        Hmid = (J * kron(Sz,Sz)) + (J/2 * kron(Sp,Sm)) + (J/2 * kron(Sm,Sp))
        Iright = Diagonal(ones((3^(N-n-1), 3^(N-n-1)))) #I_{l+2} x I_{l+3} x ... I_{N-1}
        H += kron(kron(Ileft,Hmid),Iright)
    end

    for n in 1:N
        Ileft = Diagonal(ones((3^(n-1), 3^(n-1)))) #I_0 x I_1 x ... I_{l-1}
        Hmid = -h * Sz #-h * S^z_l
        Iright = Diagonal(ones((3^(N-n), 3^(N-n)))) #I_{l+1} x I_{l+2} x ... I_{N-1}
        H += kron(kron(Ileft,Hmid),Iright)
    end
    
    return H
end

Hmat_Heis (generic function with 1 method)

In [253]:
# parameters
N = 5
J = rand()
h = rand()

# Hamiltonian matrix from MPO
H_MPO = Heis_MPO(N,J,h)
for n in 1:N
    if n == 1
        Taux = H_MPO[n][1,:,:,:]
    else
        @tensor result[i,k,l,m,o] := Taux[i,j,k] * H_MPO[n][j,l,m,o]
        Taux = result
        Taux = permutedims(Taux, (1, 3, 4, 2, 5))
        Taux = reshape(Taux, (size(Taux, 1)*size(Taux, 2), 
                              size(Taux, 3), 
                              size(Taux, 4)*size(Taux, 5)))
    end
end  

Hmat = Taux[:,1,:]

# brute-force Hamiltonian matrix
Hmat2 = Hmat_Heis(N,J,h)

# difference
H_diff = maximum(abs.(Hmat .- Hmat2))

println("difference between MPO and brute-force Hamiltonians = $H_diff")

difference between MPO and brute-force Hamiltonians = 2.220446049250313e-16


## iv) AKLT model
 
$\quad$ The AKLT model (https://doi.org/10.1103/PhysRevLett.59.799) describes a chain of $N$ $s=1$ spins, governed by the following Hamiltonian (assuming open boundary conditions):
$$
\hat{\mathcal{H}}_\text{AKLT} = \sum_{l=0}^{N-2} \left[ \hat{\vec{S}}_l \cdot \hat{\vec{S}}_{l+1} + \frac{1}{3} \left( \hat{\vec{S}}_l \cdot \hat{\vec{S}}_{l+1} \right)^2 \right].
$$

$\quad$ Straightforward manipulation leads to
$$
\hat{\mathcal{H}}_\text{AKLT} = \sum_{l=0}^{N-2} \left[ \hat{S}^z_l \hat{S}^z_{l+1} + \frac{1}{2} \hat{S}^+_l \hat{S}^-_{l+1} + \frac{1}{2} \hat{S}^-_l \hat{S}^+_{l+1} + \frac{1}{3} \hat{S}^z_l \hat{S}^z_l \hat{S}^z_{l+1} \hat{S}^z_{l+1} \right. \\
\left. + \frac{1}{6} \hat{S}^z_l \hat{S}^+_l \hat{S}^z_{l+1} \hat{S}^-_{l+1} + \frac{1}{6} \hat{S}^z_l \hat{S}^-_l \hat{S}^z_{l+1} \hat{S}^+_{l+1} + \frac{1}{6} \hat{S}^+_l \hat{S}^z_l \hat{S}^-_{l+1} \hat{S}^z_{l+1} + \frac{1}{12} \hat{S}^+_l \hat{S}^+_l \hat{S}^-_{l+1} \hat{S}^-_{l+1} \right. \\
\left. + \frac{1}{12} \hat{S}^+_l \hat{S}^-_l \hat{S}^-_{l+1} \hat{S}^+_{l+1} + \frac{1}{6} \hat{S}^-_l \hat{S}^z_l \hat{S}^+_{l+1} \hat{S}^z_{l+1} + \frac{1}{12} \hat{S}^-_l  \hat{S}^+_l \hat{S}^+_{l+1} \hat{S}^-_{l+1} + \frac{1}{12} \hat{S}^-_l \hat{S}^-_l \hat{S}^+_{l+1} \hat{S}^+_{l+1} \right].
$$

In [ ]:
function AKLT_MPO(N)
    # spin-1 operators (bottom-top)
    ## S^z_l
    Sz = zeros((3,3))
    Sz[1,1] = 1
    Sz[3,3] = -1
    ## S^+_l
    Sp = zeros((3,3))
    Sp[1,2] = sqrt(2)
    Sp[2,3] = sqrt(2)
    ## S^-_l
    Sm = zeros((3,3))
    Sm[2,1] = sqrt(2)
    Sm[3,2] = sqrt(2)
    ## I_l
    I3 = Diagonal(ones((3, 3)))
    
    # MPO Hamiltonian (left-bottom-right-top)
    ## H[l]
    Hl = zeros((14,3,14,3))
    Hl[1,:,1,:] = I3
    Hl[2,:,1,:] = Sz
    Hl[3,:,1,:] = Sm
    Hl[4,:,1,:] = Sp
    Hl[5,:,1,:] = Sz * Sz
    Hl[6,:,1,:] = Sz * Sm
    Hl[7,:,1,:] = Sz * Sp
    Hl[8,:,1,:] = Sm * Sz
    Hl[9,:,1,:] = Sm * Sm
    Hl[10,:,1,:] = Sm * Sp
    Hl[11,:,1,:] = Sp * Sz
    Hl[12,:,1,:] = Sp * Sm
    Hl[13,:,1,:] = Sp * Sp
    Hl[14,:,2,:] = Sz
    Hl[14,:,3,:] = 0.5 * Sp
    Hl[14,:,4,:] = 0.5 * Sm
    Hl[14,:,5,:] = 1/3 * Sz * Sz
    Hl[14,:,6,:] = 1/6 * Sz * Sp
    Hl[14,:,7,:] = 1/6 * Sz * Sm
    Hl[14,:,8,:] = 1/6 * Sp * Sz
    Hl[14,:,9,:] = 1/12 * Sp * Sp
    Hl[14,:,10,:] = 1/12 * Sp * Sm
    Hl[14,:,11,:] = 1/6 * Sm * Sz
    Hl[14,:,12,:] = 1/12 * Sm * Sp
    Hl[14,:,13,:] = 1/12 * Sm * Sm
    Hl[14,:,14,:] = I3
    ## H
    H = [Hl for l in 1:N]
    H[1] = Hl[14:14,:,:,:]
    H[N] = Hl[:,:,1:1,:]
    
    return H
end 

AKLT_MPO (generic function with 1 method)

In [275]:
function Hmat_AKLT(N)
    # spin-1 operators
    ## S^z_l
    Sz = zeros((3,3))
    Sz[1,1] = 1
    Sz[3,3] = -1
    ## S^+_l
    Sp = zeros((3,3))
    Sp[1,2] = sqrt(2)
    Sp[2,3] = sqrt(2)
    ## S^-_l
    Sm = zeros((3,3))
    Sm[2,1] = sqrt(2)
    Sm[3,2] = sqrt(2)
    ## I_l
    I3 = Diagonal(ones(3))
    
    # Hamiltonian matrix
    H = zeros((3^N,3^N))
    for n in 1:N-1
        Ileft = Diagonal(ones(3^(n-1))) #I_0 x I_1 x ... I_{l-1}
        Hmid = kron(Sz,Sz) #S^z_l x S^z_{l+1}
        Hmid += 0.5 * kron(Sp,Sm) #1/2 * S^+_l x S^-_{l+1}
        Hmid += 0.5 * kron(Sm,Sp) #1/2 * S^-_l x S^+_{l+1}
        Hmid += (1/3) * kron((Sz * Sz), (Sz * Sz)) #1/3 * (S^z_l * S^z_l) x (S^z_{l+1} * S^z_{l+1})
        Hmid += (1/6) * kron((Sz * Sp), (Sz * Sm)) #1/6 * (S^z_l * S^+_l) x (S^z_{l+1} * S^-_{l+1})
        Hmid += (1/6) * kron((Sz * Sm), (Sz * Sp)) #1/6 * (S^z_l * S^-_l) x (S^z_{l+1} * S^+_{l+1})
        Hmid += (1/6) * kron((Sp * Sz), (Sm * Sz)) #1/6 * (S^+_l * S^z_l) x (S^-_{l+1} * S^z_{l+1})
        Hmid += (1/12) * kron((Sp * Sp), (Sm * Sm)) #1/12 * (S^+_l * S^+_l) x (S^-_{l+1} * S^-_{l+1})
        Hmid += (1/12) * kron((Sp * Sm), (Sm * Sp)) #1/12 * (S^+_l * S^-_l) x (S^-_{l+1} * S^+_{l+1})
        Hmid += (1/6) * kron((Sm * Sz), (Sp * Sz)) #1/6 * (S^-_l * S^z_l) x (S^+_{l+1} * S^z_{l+1})
        Hmid += (1/12) * kron((Sm * Sp), (Sp * Sm)) #1/12 * (S^-_l * S^+_l) x (S^+_{l+1} * S^-_{l+1})
        Hmid += (1/12) * kron((Sm * Sm), (Sp * Sp)) #1/12 * (S^-_l * S^-_l) x (S^+_{l+1} * S^+_{l+1})
        Iright = Diagonal(ones(3^(N-n-1))) #I_{l+2} x I_{l+3} x ... I_{N-1}
        H += kron(kron(Ileft,Hmid),Iright)
    end
    return H

end


Hmat_AKLT (generic function with 1 method)

In [279]:
# parameters
N = 5

# Hamiltonian matrix from MPO
H_MPO = AKLT_MPO(N)
for n in 1:N
    if n == 1
        Taux = H_MPO[n][1,:,:,:]
    else
        @tensor result[i,k,l,m,o] := Taux[i,j,k] * H_MPO[n][j,l,m,o]
        Taux = result
        Taux = permutedims(Taux, (1, 3, 4, 2, 5))
        Taux = reshape(Taux, (size(Taux, 1)*size(Taux, 2), 
                              size(Taux, 3), 
                              size(Taux, 4)*size(Taux, 5)))
    end
end                           
Hmat = Taux[:,1,:]

# brute-force Hamiltonian matrix
Hmat2 = Hmat_AKLT(N)

# difference
H_diff = maximum(abs.(Hmat .- Hmat2))

println("difference between MPO and brute-force Hamiltonians = $H_diff")